### Doc2Vec

In [1]:
### Librerias necesarias Doc2Vec

import gensim
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

### Librerias necesarias para el preprocesamiento de texto
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

### Descargar stopwords
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.model_selection import train_test_split

### Otras librerias necesarias
import numpy as np
import pandas as pd
import re
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Cargar DataSet


In [2]:
Df = pd.read_csv("C:/Users/USER/OneDrive - universidadean.edu.co/5. NLP/Repositorio NLP/Actividad 1/reviews_booking.csv")
Df['Calificación'] = Df['Calificación'].astype(str).str.replace(',', '.').astype(float)
Df = Df.drop("Unnamed: 0", axis=1)
Df

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_21996\2155560498.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  Df = pd.read_csv("C:/Users/USER/OneDrive - universidadean.edu.co/5. NLP/Repositorio NLP/Actividad 1/reviews_booking.csv")


,País,Acomodación,Noches,Fecha hospedaje,Grupo viaje,Fecha reseña,Titulo,Calificación,Cosas Positivas,Cosas Negativas,reseña
0,Venezuela,NaN,2 noches,julio de 2023,En pareja,31 de julio de 2023,Excepcional,10.0,Fue la Mejor Opción que pudimos haber tomado e...,NaN,Sí
1,Colombia,NaN,4 noches,octubre de 2024,Persona que viaja sola,23 de octubre de 2024,Fantástica,10.0,"Ubicación, comodidades y limpieza",NaN,Sí
2,Colombia,NaN,1 noche,octubre de 2024,Persona que viaja sola,15 de octubre de 2024,Excepcional,10.0,"El lugar esta muy bien ubicado, el apartamento...",NaN,Sí
3,Colombia,NaN,1 noche,agosto de 2024,En pareja,20 de agosto de 2024,Gran apartamento en una gran ubicación.,10.0,Todo estuvo perfecto. La ubicación y la relaci...,NaN,Sí
4,Colombia,NaN,3 noches,marzo de 2024,En pareja,27 de marzo de 2024,Excepcional,10.0,"Es muy cómodo y acogedor, la atención es excel...",NaN,Sí
...,...,...,...,...,...,...,...,...,...,...,...
438208,Estados Unidos,Apartamento Suite,1 noche,septiembre de 2024,En familia,22 de octubre de 2024,Fantástico,9.0,NaN,NaN,No
438209,Colombia,Apartamento de 1 dormitorio,1 noche,octubre de 2024,Persona que viaja sola,9 de octubre de 2024,Excepcional,10.0,NaN,NaN,No
438210,México,Apartamento de 1 dormitorio,4 noches,agosto de 2024,Persona que viaja sola,6 de septiembre de 2024,Pésimo,1.0,No me pareció nada,No obstante que dos días antes de mi llegada a...,Sí
438211,Colombia,Apartamento de 1 dormitorio,2 noches,julio de 2024,En pareja,22 de julio de 2024,Excepcional,10.0,NaN,NaN,No


#### Ajuste DataFrame

In [3]:
# Crear una lista de stopwords
stop_words = set(stopwords.words('spanish'))

#Función para contar stopwords en un texto
def count_stopwords(text):
    if not isinstance(text, str):
        return 0
    words = text.split()
    return sum(1 for word in words if word.lower() in stop_words)

## Dataframe positivos
# Contar stopwords en la columna 'Cosas Positivas'
stopword_counts = Df['Cosas Positivas'].apply(count_stopwords)

# Filtrar filas con menos de 5 stopwords
Df_positivo = Df[stopword_counts >= 15]

# Filtrar filas donde 'Cosas Positivas' tenga más de #### palabra
Df_positivo = Df_positivo[Df_positivo['Cosas Positivas'].str.split().str.len() > 30]

# Filtrar filas donde 'Calificación' sea mayor a 9
Df_positivo = Df_positivo[Df_positivo['Calificación'] > 9]


## Dataframe negativos
# Contar stopwords en la columna 'Cosas Positivas'
stopword_counts = Df['Cosas Negativas'].apply(count_stopwords)

# Filtrar filas con menos de 5 stopwords
Df_negativo = Df[stopword_counts >= 20]

# Filtrar filas donde 'Cosas Positivas' tenga más de #### palabra
Df_negativo = Df_negativo[
    (Df_negativo['Cosas Negativas'].str.split().str.len() > 20) &
    (Df_negativo['Cosas Negativas'].str.split().str.len() <= 70)
]

# Filtrar filas donde 'Calificación' sea menor
Df_negativo = Df_negativo[Df_negativo['Calificación'] < 7]

In [4]:
# Crear una nueva columna para identificar los comentarios como positivos
Df_positivo['Etiqueta'] = 'positivo'

# Crear una nueva columna para identificar los comentarios como negativos
Df_negativo['Etiqueta'] = 'negativo'

# Renombrar la columna de los comentarios en ambos DataFrames para que sean iguales
Df_positivo = Df_positivo.rename(columns={'Cosas Positivas': 'Comentario'})
Df_negativo = Df_negativo.rename(columns={'Cosas Negativas': 'Comentario'})

# Combinar los dos DataFrames en uno solo
Df_unificado = pd.concat([Df_positivo[['Comentario', 'Etiqueta']], Df_negativo[['Comentario', 'Etiqueta']]])

# Resetear el índice del DataFrame combinado
Df_unificado  = Df_unificado .reset_index(drop=True)

Df_unificado

,Comentario,Etiqueta
0,Fue la Mejor Opción que pudimos haber tomado e...,positivo
1,Me gusto Poder comprar los alimentos a mi gust...,positivo
2,"La calidad de personas es maravilloso, desde q...",positivo
3,La atención como me recibieron es muy buena po...,positivo
4,"El hotel es muy acogedor, está ubicado en una ...",positivo
...,...,...
10742,"Nada, no pude alojarme porque simplemente canc...",negativo
10743,Ya habíamos ido anteriormente y nos valió $150...,negativo
10744,"Publicidad engañosa, ofrecen servicios que no ...",negativo
10745,"Tv sin control remoto., baños con pisos muy pe...",negativo


In [5]:
# Dividir el dataset en entrenamiento y prueba (sin variable objetivo)
train_df, test_df = train_test_split(Df_unificado, test_size=0.2, random_state=42)

# Verificar las dimensiones
print("Shape of X_train:", train_df.shape)
print("Shape of X_test:", test_df.shape)

train_df

Shape of X_train: (8597, 2)
Shape of X_test: (2150, 2)


,Comentario,Etiqueta
4335,Tuve una excelente experiencia. El hotel está ...,positivo
353,"El personal de lujo del hotel, Jesús, Junior f...",positivo
456,"Excelente atención, los recepcionistas te dan ...",positivo
5317,El desayuno es variado y abundante el buffet.\...,positivo
239,Estuve con mi familia y nos atendieron súper b...,positivo
...,...,...
5734,La calidad humana con la que fui acogida y tra...,positivo
5191,El desayuno es espectacular; su personal muy a...,positivo
5390,El hostal es muy agreable. Tiene areas differe...,positivo
860,"La atención de doña Gladis es espectacular, to...",positivo


In [6]:
print(type(Df_unificado))
print(Df_unificado.keys())  # Si es un diccionario, mostrará las claves disponibles


<class 'pandas.core.frame.DataFrame'>
Index(['Comentario', 'Etiqueta'], dtype='object')


In [7]:
# Verificar las dimensiones
print("Shape of train data: ", train_df.shape)
print("Shape of test data: ", test_df.shape)

# Mostrar los primeros registros del conjunto de entrenamiento
train_df.head()

Shape of train data:  (8597, 2)
Shape of test data:  (2150, 2)


,Comentario,Etiqueta
4335,Tuve una excelente experiencia. El hotel está ...,positivo
353,"El personal de lujo del hotel, Jesús, Junior f...",positivo
456,"Excelente atención, los recepcionistas te dan ...",positivo
5317,El desayuno es variado y abundante el buffet.\...,positivo
239,Estuve con mi familia y nos atendieron súper b...,positivo


In [8]:
test_df.head()

,Comentario,Etiqueta
668,"Genial el lugar, la ubicación es muy buena, un...",positivo
4874,La ubicación y la atención de todo el personal...,positivo
7410,El hotel esta deteriorado. Los baños en mal es...,negativo
8274,La habitación no cumplió con las expectativas ...,negativo
8197,La nevera sonaba como un motor de avión y esta...,negativo


#### Preprocesamiento

In [9]:
########## Preprocesamiento de texto ###############

def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    return text

# Aplicando la función de preprocesamiento a los datos de entrenamiento y prueba

train_df['text_pre'] = train_df['Comentario'].apply(preprocess_text)

test_df['text_pre'] = test_df['Comentario'].apply(preprocess_text)

train_df.head()

,Comentario,Etiqueta,text_pre
4335,Tuve una excelente experiencia. El hotel está ...,positivo,tuve una excelente experiencia. el hotel está ...
353,"El personal de lujo del hotel, Jesús, Junior f...",positivo,"el personal de lujo del hotel, jesús, junior f..."
456,"Excelente atención, los recepcionistas te dan ...",positivo,"excelente atención, los recepcionistas te dan ..."
5317,El desayuno es variado y abundante el buffet.\...,positivo,el desayuno es variado y abundante el buffet.\...
239,Estuve con mi familia y nos atendieron súper b...,positivo,estuve con mi familia y nos atendieron súper b...


## 4. Modelo de Clasificación de Texto

Una vez que hemos preprocesado los textos, podemos aplicar un modelo de clasificación no supervisado para agruparlos en categorías o clusters. En este caso, utilizaremos el algoritmo de clustering KMeans para agrupar los textos en clusters. AUnque primero debemos convertir los textos en vectores numéricos utilizando embeddings de palabras.


In [10]:
################## Entrenamiento de Word2Vec ####################

# Tokenizando el texto

train_df['text_tokens'] = train_df['Comentario'].apply(lambda x: x.split())
test_df['text_tokens'] = test_df['Comentario'].apply(lambda x: x.split())

train_df.head()

,Comentario,Etiqueta,text_pre,text_tokens
4335,Tuve una excelente experiencia. El hotel está ...,positivo,tuve una excelente experiencia. el hotel está ...,"[Tuve, una, excelente, experiencia., El, hotel..."
353,"El personal de lujo del hotel, Jesús, Junior f...",positivo,"el personal de lujo del hotel, jesús, junior f...","[El, personal, de, lujo, del, hotel,, Jesús,, ..."
456,"Excelente atención, los recepcionistas te dan ...",positivo,"excelente atención, los recepcionistas te dan ...","[Excelente, atención,, los, recepcionistas, te..."
5317,El desayuno es variado y abundante el buffet.\...,positivo,el desayuno es variado y abundante el buffet.\...,"[El, desayuno, es, variado, y, abundante, el, ..."
239,Estuve con mi familia y nos atendieron súper b...,positivo,estuve con mi familia y nos atendieron súper b...,"[Estuve, con, mi, familia, y, nos, atendieron,..."


In [11]:
# Entrenando el modelo Doc2Vec

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(train_df['text_tokens'])]

# Definiendo el modelo Doc2Vec

model = Doc2Vec(documents, vector_size=100, window=2, min_count=1, workers=4)

# Guardando el modelo entrenado

model.save("doc2vec.model")


In [12]:
# Cargando el modelo entrenado

model = Doc2Vec.load("doc2vec.model")


# Obteniendo las palabras más similares a una palabra

model.wv.most_similar('excelente')


[('excepcional', 0.8740154504776001),
 ('Excelente', 0.8703814744949341),
 ('increíble', 0.8678728938102722),
 ('espectacular', 0.8600459098815918),
 ('inigualable', 0.8594076037406921),
 ('buena', 0.8437235355377197),
 ('gran', 0.8370811343193054),
 ('excepcional,', 0.8295101523399353),
 ('pésima', 0.8288291096687317),
 ('excelente.', 0.8236297965049744)]

In [13]:
# Obteniendo la similitud entre dos palabras

model.wv.similarity('hotel', 'hospedaje')


0.8326478

In [14]:
#### Hacemos un clustering de los tweets con KMeans

from sklearn.cluster import KMeans

# Obteniendo los vectores de los tweets

vectors = [model.infer_vector(doc) for doc in train_df['text_tokens']]

# Definiendo el modelo KMeans

kmeans = KMeans(n_clusters=5, random_state=0)

# Entrenando el modelo KMeans

kmeans.fit(vectors)

# Obteniendo las etiquetas de los clusters

train_df['cluster'] = kmeans.labels_

# Explorando los clusters

train_df['cluster'].value_counts()



Exception in thread Thread-54 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\USER\miniconda3\envs\entorno_npl_ean_py3.10\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\USER\miniconda3\envs\entorno_npl_ean_py3.10\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\USER\miniconda3\envs\entorno_npl_ean_py3.10\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\USER\miniconda3\envs\entorno_npl_ean_py3.10\lib\subprocess.py", line 1515, in _readerthread
    buffer.append(fh.read())
  File "c:\Users\USER\miniconda3\envs\entorno_npl_ean_py3.10\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 16: invalid start byte
c:\Users\USER\miniconda3\envs\entorno_npl_ean_py3.10\lib\site-packages\joblib\externals\lok

cluster
4    2993
2    2153
0    1897
3    1349
1     205
Name: count, dtype: int64

In [15]:
### Veamos los tweets de un cluster

pd.set_option('display.max_colwidth', None)

train_df[train_df['cluster'] == 0]['Comentario'].head(10)

6749     El lugar que publican en Booking no corresponde al real, la habitación que se había seleccionado no correspondía, era una cama doble y me iban a asignar tres camas sencillas. La habitación asignada después no tenía agua la sister a del baño, no se pudo dormir, toda la noche se activaba una máquina que perturbaba el sueño. El desayuno ofrecido pésimo.
5155                                                                                                    Muy limpio,muy amables, te ayudan en lo que puedan, puedes pedir plancha sin costo, el desayuno es opcional por pago extra y te preguntan que quieres, sería bueno que pusieran letrero con el nombre para evitar perderse.\nSeguridad muy curiosa pero efectiva.
10440                                                              No tiene servicio de restaurante,el desayuno es regular,no tiene refrigerador en las habitaciones, aparte las instalaciones eléctricas están mal distribuidas y debes salir de la cama para encender o apagar.\nN

In [16]:
### Veamos el centroide de cada cluster

for i in range(5):
    print("Cluster ", i)
    print(model.wv.most_similar(positive=[kmeans.cluster_centers_[i]], topn=10))
    print("\n\n")

Cluster  0
[('ja', 0.9710492491722107), ('nisiquiera', 0.9638456702232361), ('impuestos.', 0.9635998606681824), ('previamente', 0.9595365524291992), ('costaba', 0.9590597152709961), ('ASI.GRATO', 0.9589343667030334), ('dolares', 0.9588800668716431), ('más,', 0.9583945274353027), ('pedi', 0.9578806757926941), ('excusas,', 0.9566056132316589)]



Cluster  1
[('bom', 0.9844905734062195), ('confortáveis,', 0.9830412268638611), ('LLEGAR', 0.9829716682434082), ('POR', 0.982852041721344), ('LOS', 0.9826971292495728), ('lieux', 0.981971263885498), ('PARTE', 0.981766402721405), ('chuveiro', 0.9810115694999695), ('était', 0.9806631803512573), ('ME', 0.9803190231323242)]



Cluster  2
[('sirve,', 0.9404832720756531), ('correctamente', 0.9394145011901855), ('elevador', 0.9347614645957947), ('jabón.', 0.9344375133514404), ('apenas', 0.9315478205680847), ('pelo,', 0.9295362830162048), ('mantenimiento,', 0.9294677972793579), ('entonces', 0.9293555617332458), ('higienico', 0.9286630153656006), ('pelos